In [ ]:
!tar xf image.tar

tar: image.tar: Cannot open: No such file or directory
tar: Error is not recoverable: exiting now


In [ ]:
import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

norm_layer = nn.InstanceNorm2d


class ResidualBlock(nn.Module):
    def __init__(self, in_features):
        super(ResidualBlock, self).__init__()

        conv_block = [
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            norm_layer(in_features),
            nn.ReLU(inplace=True),
            nn.ReflectionPad2d(1),
            nn.Conv2d(in_features, in_features, 3),
            norm_layer(in_features),
        ]

        self.conv_block = nn.Sequential(*conv_block)

    def forward(self, x):
        return x + self.conv_block(x)


class Generator(nn.Module):
    def __init__(self, input_nc, output_nc, n_residual_blocks=9, sigmoid=True):
        super(Generator, self).__init__()

        model0 = [
            nn.ReflectionPad2d(3),
            nn.Conv2d(input_nc, 64, 7),
            norm_layer(64),
            nn.ReLU(inplace=True),
        ]
        self.model0 = nn.Sequential(*model0)

        model1 = []
        in_features = 64
        out_features = in_features * 2
        for _ in range(2):
            model1 += [
                nn.Conv2d(in_features, out_features, 3, stride=2, padding=1),
                norm_layer(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
            out_features = in_features * 2
        self.model1 = nn.Sequential(*model1)

        model2 = []
        for _ in range(n_residual_blocks):
            model2 += [ResidualBlock(in_features)]
        self.model2 = nn.Sequential(*model2)

        model3 = []
        out_features = in_features // 2
        for _ in range(2):
            model3 += [
                nn.ConvTranspose2d(
                    in_features, out_features, 3, stride=2, padding=1, output_padding=1
                ),
                norm_layer(out_features),
                nn.ReLU(inplace=True),
            ]
            in_features = out_features
            out_features = in_features // 2
        self.model3 = nn.Sequential(*model3)

        model4 = [nn.ReflectionPad2d(3), nn.Conv2d(64, output_nc, 7)]
        if sigmoid:
            model4 += [nn.Sigmoid()]

        self.model4 = nn.Sequential(*model4)

    def forward(self, x, cond=None):
        out = self.model0(x)
        out = self.model1(out)
        out = self.model2(out)
        out = self.model3(out)
        out = self.model4(out)

        return out

device = "cpu"
model1 = Generator(3, 1, 3)
model1.load_state_dict(torch.load("../model2.pth", map_location=torch.device("cuda")))
model1 = model1.to(device)
model1.eval()

target_size = 256
import requests
import random
import time
import cv2
import numpy as np

def predict(path, model):
    input_img = cv2.imread(path)

    input_img = Image.open(path)
    transform = transforms.Compose(
       [transforms.ToPILImage(), transforms.Resize(1080, 1080), transforms.ToTensor()]
    )

    input_img = transform(input_img)
    input_img = torch.unsqueeze(input_img, 0)
    drawing = 0
    input_img = input_img.to(device)
    with torch.no_grad():
        drawing = model(input_img)[0].detach()

    drawing = transforms.ToPILImage()(drawing)

    # constant by which each pixel is divided
    drawing = drawing.point(lambda i: darken_pixel(i))

    im_output = drawing.resize((target_size, target_size))
    return im_output


def darken_pixel(pixel):
    constant = 2.0
    if pixel < 200:
        return pixel / constant
    else:
        return pixel


ModuleNotFoundError: ignored

In [ ]:
from tqdm.notebook import tqdm
import os
types = ["Apple", "Gmail", "Facebook", "Google, ""Twitter", "Windows"]
for type_ in types:
  for idx, image_path in tqdm(enumerate(os.listdir("./image/" + type_))):
    prediction = predict("./image/" + type_ + "/" + image_path)
    prediction.save("./sketches/" + type_ + "/" + image_path)

0it [00:00, ?it/s]

0it [00:00, ?it/s]